In [23]:
import ee
ee.Initialize(project='ee-nigeria-disease-project')

In [24]:
import geemap  # For direct local exports
import requests  # For URL-based pulls
import numpy as np
import pandas as pd

#### CHIRPS Rainfall (Precipitation: Key for Cholera/Malaria)

In [ ]:
# Nigeria bounds
nigeria = ee.Geometry.Rectangle([3, 4, 15, 14])

try:
    # Load daily CHIRPS — CORRECTED PATH (slash, not hyphen)
    chirps = ee.ImageCollection('UCSBC-HG/CHIRPS/DAILY') \
        .filterDate('2018-01-01', '2024-12-31') \
        .filterBounds(nigeria) \
        .select('precipitation')

    # Verify load (expect ~2500+ images)
    filtered_size = chirps.size().getInfo()
    print(f"CHIRPS loaded! Filtered images: {filtered_size}")

    # Generate list of all month starts (2018–2024)
    years = ee.List.sequence(2018, 2024)
    month_starts = years.map(lambda y: ee.List.sequence(1, 12).map(lambda m: ee.Date.fromYMD(y, m, 1)))

    # Function to compute mean for a single month
    def compute_monthly_mean(month_start):
        start = ee.Date(month_start)
        end = start.advance(1, 'month')
        monthly_coll = chirps.filterDate(start, end)
        # Mean daily precip for the month
        mean_img = monthly_coll.mean().clip(nigeria)
        # Set properties for band naming (e.g., '2018_01')
        year = start.get('year')
        month = start.get('month').format('%02d')
        band_name = ee.String(year).cat('_').cat(month)
        return mean_img.rename(band_name).set('system:time_start', start.millis())

    # Create multi-band image
    monthly_means = ee.ImageCollection(month_starts.flatten().map(compute_monthly_mean)).toBands()

    # Export to Google Drive
    task = ee.batch.Export.image.toDrive(
        image=monthly_means,
        description='chirps_nigeria_monthly_multi_band_2018_2024',
        folder='GEE_Exports',
        region=nigeria,
        scale=5566,  # ~5 km
        crs='EPSG:4326',
        maxPixels=1e10,  # Increased for multi-band
        fileFormat='GeoTIFF',
        skipEmptyTiles=True
    )
    task.start()

    print("CHIRPS multi-band export started—check https://code.earthengine.google.com/tasks")
    print("File: GEE_Exports/chirps_nigeria_monthly_multi_band_2018_2024.tif (~84 bands, 100-200MB)")

except ee.EEException as e:
    print("CHIRPS still failing:", str(e))
    print("Falling back to GPM multi-band version...")
    # GPM fallback (similar structure, daily mm/hr *24 to mm/day)
    gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
        .filterDate('2018-01-01', '2024-12-31') \
        .filterBounds(nigeria) \
        .select('precipitationCal')
    
    def compute_gpm_monthly(month_start):
        start = ee.Date(month_start)
        end = start.advance(1, 'month')
        monthly_coll = gpm.filterDate(start, end)
        mean_img = monthly_coll.map(lambda img: img.multiply(24)).mean().clip(nigeria)  # mm/day
        year = start.get('year')
        month = start.get('month').format('%02d')
        band_name = ee.String('GPM_').cat(year).cat('_').cat(month)
        return mean_img.rename(band_name).set('system:time_start', start.millis())
    
    gpm_monthly = ee.ImageCollection(month_starts.flatten().map(compute_gpm_monthly)).toBands()
    
    task_gpm = ee.batch.Export.image.toDrive(
        image=gpm_monthly,
        description='gpm_nigeria_monthly_multi_band_2018_2024',
        folder='GEE_Exports',
        region=nigeria,
        scale=10000,  # GPM native ~10km
        crs='EPSG:4326',
        maxPixels=1e10,
        fileFormat='GeoTIFF',
        skipEmptyTiles=True
    )
    task_gpm.start()
    print("GPM fallback started: https://code.earthengine.google.com/tasks")

CHIRPS still failing: ImageCollection.load: ImageCollection asset 'UCSBC-HG/CHIRPS/DAILY' not found (does not exist or caller does not have access).
Falling back to GPM multi-band version...
GPM fallback started: https://code.earthengine.google.com/tasks


In [34]:
import rasterio
with rasterio.open(r'C:\Users\Hp\Documents\capstone_project\data\raw\gpm_nigeria_monthly_multi_band_2018_2024.tif') as src:
    data = src.read(1)
    print(f"Shape: {data.shape} | Min/Max (mm/month): {data.min():.1f}/{data.max():.1f}")

Shape: (113, 134) | Min/Max (mm/month): nan/nan


##### NDVI (Vegetation: MODIS-Derived)

In [35]:
# Load MODIS NDVI
modis = ee.ImageCollection('MODIS/006/MOD13A2') \
    .filterDate('2018-01-01', '2024-12-31') \
    .filterBounds(nigeria) \
    .select('NDVI')

ndvi_mean = modis.mean().multiply(0.0001)  # Scale to 0-1

task_ndvi = ee.batch.Export.image.toDrive(
    image=ndvi_mean,
    description='ndvi_nigeria_mean_2018_2024_2',
    folder='geoai_project',
    region=nigeria,
    scale=1000,  # 1km; resample later
    crs='EPSG:4326',
    maxPixels=1e9
)
task_ndvi.start()
print(f"NDVI task: {task_ndvi.id}")

NDVI task: GUHHY6RKN5WLVM3FJZ5F2KP2


##### Temperature (NASA POWER: Air Temp)

In [36]:
# NASA POWER temp via GEE (monthly T2M)
power = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
    .filterDate('2018-01-01', '2024-12-31') \
    .filterBounds(nigeria) \
    .select('Tair_f_inst')  # Near-surface air temp (K; convert to °C later)

temp_mean = power.reduce(ee.Reducer.mean()).subtract(273.15)  # K to °C

task_temp = ee.batch.Export.image.toDrive(
    image=temp_mean,
    description='temp_nigeria_monthly_mean_2018_2024',
    folder='geoai_project',
    region=nigeria,
    scale=5566,
    crs='EPSG:4326',
    maxPixels=1e9
)
task_temp.start()
print(f"Temp task: {task_temp.id}")

Temp task: FP4MCZPKQNDIKMW6U5GR5HYH


##### Elevation (SRTM DEM: Static Terrain)

In [38]:
srtm = ee.Image('USGS/SRTMGL1_003').clip(nigeria).select('elevation')

task_srtm = ee.batch.Export.image.toDrive(
    image=srtm,
    description='srtm_nigeria_elevation',
    folder='geoai_project',
    region=nigeria,
    scale=30,
    crs='EPSG:4326',
    maxPixels=1e10
)
task_srtm.start()
print(f"SRTM task: {task_srtm.id}")

SRTM task: SOQHHLQGTCCG6HXT6XYHMKRL


##### Population Density (WorldPop/GRID3)
Download gridded TIFF from worldpop.org (100m res, 2020)

##### Roads & Water Bodies (OpenStreetMap)

In [48]:
import requests
import zipfile
import geopandas as gpd
import os

# Create data/raw/
os.makedirs('data/raw', exist_ok=True)

# Download URL for Nigeria shapefiles (full OSM extract, ~200MB ZIP)
url = 'https://download.geofabrik.de/africa/nigeria-latest-free.shp.zip'
zip_path = 'data/raw/nigeria_osm_shapefiles.zip'

# Fetch and save ZIP
print("Downloading Nigeria OSM shapefiles (~200MB, 1-2 min)...")
response = requests.get(url, stream=True)
with open(zip_path, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

# Extract ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('data/raw/')

# Load and filter roads (highways/primary/secondary/etc.)
roads_gdf = gpd.read_file('data/raw/gis_osm_roads_free_1.shp')  # Main roads layer
roads_gdf = roads_gdf[roads_gdf['fclass'].isin(['primary', 'secondary', 'tertiary', 'road', 'trunk'])]  # Filter drivable
roads_gdf.to_file('data/raw/osm_roads_nigeria.geojson', driver='GeoJSON')

# Load and filter water (rivers, waterbodies)
water_gdf = gpd.read_file('data/raw/gis_osm_waterways_free_1.shp')  # Rivers
water_rivers = water_gdf[water_gdf['fclass'].isin(['river', 'canal', 'stream'])]
water_bodies = gpd.read_file('data/raw/gis_osm_water_a_free_1.shp')  # Lakes/reservoirs
water_bodies = water_bodies[water_bodies['fclass'].isin(['water', 'reservoir', 'lake'])]
water_gdf = gpd.pd.concat([water_rivers, water_bodies], ignore_index=True)  # Combine
water_gdf.to_file('data/raw/osm_water_nigeria.geojson', driver='GeoJSON')

# Cleanup ZIP
os.remove(zip_path)

print("OSM vectors saved from Geofabrik!")
print(f"Roads: {len(roads_gdf)} segments | Water: {len(water_gdf)} features")

OSM vectors saved from Geofabrik!
Roads: 41602 segments | Water: 43649 features
